In [ ]:
import numpy as np
import pandas as pd
import operator
import random
import matplotlib.pyplot as plt
import time

####Benchmark functions

**Holder Table**

In [ ]:
def holder_table(x):
    return -1 * np.abs(np.sin(x[0]) * np.cos(x[1]) * np.exp(np.abs(1 - np.sqrt(x[0]**2 + x[1]**2) / np.pi)))

**Ackley**

In [ ]:
def ackley(x):
    a = 20
    b = 0.2
    c = 2*np.pi
    n = len(x)
    return -a * np.exp(-b * np.sqrt(1/n * (x[0]**2 + x[1]**2))) - np.exp(1/n * (np.cos(c * x[0]) + np.cos(c * x[1]))) + np.e + a

**Rosenbrock**

In [ ]:
def rosenbrock(x):
    ans=0.0
    for i in range(min(len(x),16)):
        ans+=(100.0*(x[i]-x[i]**2)**2 + (1-x[i])**2)
    return ans

**Griewank**

In [ ]:
def griewank(x):
    n = len(x)
    sum_term = (x[0]**2 + x[1]**2)/ 4000.0
    prod_term = np.cos(x[0]/np.sqrt(1)) * np.cos(x[1]/np.sqrt(2))
    value = 1. + sum_term - prod_term

    return value

**Rastrigin**

In [ ]:
def rastrigin(x):
    value = 0
    for i in range(len(x)):
        value += (x[i]**2 - 10 * np.cos(2 * np.pi * x[i]))
    value += 10 * (len(x))
    return value

**Schaffer**

In [ ]:
def schaffer_no_2(x):
    num = np.sin(np.sqrt(x[0]**2 + x[1]**2))**2 - 0.5
    denom = 1 + 0.001 * (x[0]**2 + x[1]**2)**2
    value = 0.5 + num / denom
    return value

####**Firefly Algorithm**

In [ ]:
class Firefly():
    def __init__(self, alpha, beta, gamma, upper_boundary, lower_boundary, function_dimension):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.__intensity = None
        self.lower_boundary = lower_boundary
        self.upper_boundary = upper_boundary
        self.__position = np.array([random.uniform(self.lower_boundary, self.upper_boundary) for _ in range(function_dimension)])

    @property
    def intensity(self):
        return self.__intensity

    @property
    def position(self):
        return self.__position

    @position.setter
    def position(self, value):
        self.__position = value

    def move_towards(self, better_position):
        distance = np.linalg.norm(self.__position - better_position)
        self.__position = self.__position + self.beta * np.exp(-self.gamma * (distance ** 2)) * (better_position - self.__position) + self.alpha * (random.uniform(0, 1) - 0.5)
        self.check_boundaries()

    def random_walk(self, area):
        self.__position = np.array([random.uniform(cord - area, cord + area) for cord in self.__position])

    def update_intensity(self, func):
        self.__intensity = -1 * func(self.__position)

    def check_boundaries(self):
        for i, cord in enumerate(self.__position):
            if cord < self.lower_boundary:
                self.__position[i] = self.lower_boundary
            elif cord > self.upper_boundary:
                self.__position[i] = self.upper_boundary
            else:
                self.__position[i] = cord

In [ ]:
class FireflyProblem():
    def __init__(self, function, firefly_number, upper_boundary, lower_boundary, alpha, beta, gamma, iteration_number, interval):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.function_dimension = 2
        self.upper_boundary = upper_boundary
        self.lower_boundary = lower_boundary
        self.iteration_number = iteration_number
        self.fireflies = [Firefly(self.alpha, self.beta, self.gamma, self.upper_boundary, self.lower_boundary, self.function_dimension) for _ in range(firefly_number)]
        self.function = function
        self.interval = interval
        self.best = None
        self.cost = []

        for firefly in self.fireflies:
            firefly.update_intensity(self.function)

    def run(self):
        for _ in range(self.iteration_number):
            self.step()

    def step(self):
        self.fireflies.sort(key=operator.attrgetter('intensity'), reverse=True)

        for i in self.fireflies:
            for j in self.fireflies:
                if j.intensity > i.intensity:
                    i.move_towards(j.position)
                    i.update_intensity(self.function)

        if not self.best or self.fireflies[0].intensity > self.best:
            self.best = self.fireflies[0].intensity

        self.cost.append(-self.best)
        self.fireflies[0].random_walk(0.1)
        self.fireflies[0].update_intensity(self.function)

In [ ]:
def run_firefly_simulations(optimizer, num_simulations, max_iter, lower_boundary, upper_boundary, firefly_number, alpha, beta, gamma, interval):
    results_df = pd.DataFrame()

    for _ in range(num_simulations):
        firefly_problem = FireflyProblem(optimizer, firefly_number, lower_boundary, upper_boundary, alpha, beta, gamma, max_iter, interval)
        firefly_problem.run()
        column_name = f'Simulation_{_ + 1}'
        results_df[column_name] = firefly_problem.cost

    return results_df

#### Tests

**Ackley**

In [ ]:
ackley_test_results = []
A_runtime_list = []

for i in range(0,100):

    A_start = time.time()
    ackley_test_results.append(run_firefly_simulations(ackley, 1, 100, -10, 10, 50, 0.2,1,1, 500))
    A_end = time.time()
    A_runtime_list.append(A_end-A_start)

print(f'FA Ackley time = {np.mean(A_runtime_list)}')

FA Ackley time = 0.2999542737007141


In [ ]:
ackley_test_results

[    Simulation_1
 0       6.271313
 1       6.271313
 2       6.189390
 3       6.145267
 4       6.044273
 ..           ...
 95      5.363033
 96      5.363033
 97      5.363033
 98      5.363033
 99      5.363033
 
 [100 rows x 1 columns],
     Simulation_1
 0       7.570021
 1       7.271613
 2       7.186803
 3       7.186803
 4       7.158101
 ..           ...
 95      5.434554
 96      5.434554
 97      5.434554
 98      5.434554
 99      5.434554
 
 [100 rows x 1 columns],
     Simulation_1
 0       7.727846
 1       7.721010
 2       7.721010
 3       7.629233
 4       7.629233
 ..           ...
 95      6.266360
 96      6.266360
 97      6.266360
 98      6.266360
 99      6.266360
 
 [100 rows x 1 columns],
     Simulation_1
 0       5.641764
 1       5.224088
 2       4.964713
 3       4.419398
 4       4.419398
 ..           ...
 95      4.360968
 96      4.360968
 97      4.360968
 98      4.360968
 99      4.360968
 
 [100 rows x 1 columns],
     Simulation_1
 0       3

In [ ]:
no_agents = [25,50,75]

df_A_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_A_tuning[df_name] = run_firefly_simulations(ackley, 100, 100, -10, 10, n, 0.2,1,1, 500)

# Save results to Excel file
with pd.ExcelWriter('FA_Ackley_tuning.xlsx') as writer:
    for df_name, df in df_A_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Holder Table**

In [ ]:
holder_test_results = []
HT_runtime_list = []

for i in range(0,100):

    HT_start = time.time()
    holder_test_results.append(run_firefly_simulations(holder_table, 1, 100, -10, 10, 50, 0.2,1,1, 500))
    HT_end = time.time()
    HT_runtime_list.append(HT_end-HT_start)

print(f'FA Holder time = {np.mean(HT_runtime_list)}')

FA Holder time = 0.26222819805145264


In [ ]:
no_agents = [25,50,75]

df_HT_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_HT_tuning[df_name] = run_firefly_simulations(holder_table, 100, 100, -10, 10, n, 0.2,1,1, 500)

# Save results to Excel file
with pd.ExcelWriter('FA_HolderTable_tuning.xlsx') as writer:
    for df_name, df in df_HT_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Rosenbrock**

In [ ]:
rosenbrock_test_results = []
Ros_runtime_list = []

for i in range(0,100):

    Ros_start = time.time()
    rosenbrock_test_results.append(run_firefly_simulations(rosenbrock, 1, 100, -5, 10, 50, 0.2,1,1, 500))
    Ros_end = time.time()
    Ros_runtime_list.append(Ros_end-Ros_start)

print(f'FA Rosenbrock time = {np.mean(Ros_runtime_list)}')

FA Rosenbrock time = 0.46513765573501586


In [ ]:
no_agents = [25,50,75]

df_Ros_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_Ros_tuning[df_name] = run_firefly_simulations(rosenbrock, 100, 100, -5, 10, n, 0.2,1,1, 500)

# Save results to Excel file
with pd.ExcelWriter('FA_Rosenbrock_tuning.xlsx') as writer:
    for df_name, df in df_Ros_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Griewank**

In [ ]:
griewank_test_results = []
G_runtime_list = []

for i in range(0,100):

    G_start = time.time()
    griewank_test_results.append(run_firefly_simulations(griewank, 1, 100, -100, 100, 50, 0.2,1,1, 500))
    G_end = time.time()
    G_runtime_list.append(G_end-G_start)

print(f'FA Griewank time = {np.mean(G_runtime_list)}')

FA Griewank time = 0.30146286487579343


In [ ]:
no_agents = [25,50,75]

df_G_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_G_tuning[df_name] = run_firefly_simulations(griewank, 100, 100, -100, 100, n, 0.2,1,1, 500)

# Save results to Excel file
with pd.ExcelWriter('FA_Griewank_tuning.xlsx') as writer:
    for df_name, df in df_G_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Rastrigin**

In [ ]:
rastrigin_test_results = []
R_runtime_list = []

for i in range(0,100):

    R_start = time.time()
    rastrigin_test_results.append(run_firefly_simulations(rastrigin, 1, 100, -5.12, 5.12, 50, 0.2,1,1, 500))
    R_end = time.time()
    R_runtime_list.append(R_end-R_start)

print(f'FA Rastrigin time = {np.mean(R_runtime_list)}')

FA Rastrigin time = 0.26647424697875977


In [ ]:
no_agents = [25,50,75]

df_Ras_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_Ras_tuning[df_name] = run_firefly_simulations(rastrigin, 100, 100, -5.12, 5.12, n, 0.2,1,1, 500)

# Save results to Excel file
with pd.ExcelWriter('FA_Rastrigin_tuning.xlsx') as writer:
    for df_name, df in df_Ras_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Schaffer No.2**

In [ ]:
schaffer_test_results = []
Sc_runtime_list = []

for i in range(0,100):

    Sc_start = time.time()
    schaffer_test_results.append(run_firefly_simulations(schaffer_no_2, 1, 100, -100, 100, 50, 0.2,1,1, 500))
    Sc_end = time.time()
    Sc_runtime_list.append(Sc_end-Sc_start)

print(f'FA Schaffer time = {np.mean(Sc_runtime_list)}')

FA Schaffer time = 0.22862247943878175


In [ ]:
no_agents = [25,50,75]

df_Sc_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_Sc_tuning[df_name] = run_firefly_simulations(schaffer_no_2, 100, 100, -100, 100, n, 0.2,1,1, 500)

# Save results to Excel file
with pd.ExcelWriter('FA_SchafferNo2_tuning.xlsx') as writer:
    for df_name, df in df_Sc_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

####Summary

In [ ]:
import pandas as pd

data = {
    'Function': ['Ackley', 'Holder Table', 'Griewank', 'Rastrigin'],
    'SFSA': [np.mean(A_runtime_list),np.mean(HT_runtime_list),np.mean(G_runtime_list),np.mean(R_runtime_list)]
}

df = pd.DataFrame(data, columns=['Function', 'SFSA'])

df

,Function,SFSA
0,Ackley,0.299954
1,Holder Table,0.262228
2,Griewank,0.301463
3,Rastrigin,0.266474
